In [2]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import *
import pyspark



In [3]:
from transaction import *
from user import *

In [4]:
spark = (SparkSession
         .builder
         .appName("firstTest")
         .config('spark.jars.packages', "io.delta:delta-core_2.12:2.3.0,io.delta:delta-storage:2.3.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
         .config('spark.sql.extension', 'io.delta.sql.DeltaSparkSessionExtension')
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .enableHiveSupport()
         .getOrCreate())

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

23/05/06 16:15:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[key: string, value: string]

In [5]:
checkpoint_dir = "/home/bcturner/instantPaymentProcessingSystem/main/data/central_processor_checkpoints"

In [7]:
input_stream = (
    spark
    .readStream 
    .format('kafka')
    .option('kafka.bootstrap.servers', 'localhost:9092')
    .option('subscribe', 'central_topic')
    .option('earliestOffset', 'latest')
    .load()
    .selectExpr("CAST (value AS STRING)")
    
    #Parse Kafka Message
    .select(
        split(col('value'), ",").getItem(3).alias("instId"),
        "value")
    .withColumn("topic", 
                concat(lit('institutionId_'),col("instId").cast(types.StringType()) )  
               )
    
    #Write to Correct Kafka Topic
    .writeStream
    .format('kafka')
    .outputMode('append')
    .option('kafka.bootstrap.servers', 'localhost:9092')
    .option('checkpointLocation', "/home/bcturner/instantPaymentProcessingSystem/main/data/central_processor_checkpoints/")
    .start()
    .awaitTermination()
)

23/05/06 16:15:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 2
-------------------------------------------
+------+--------------------+------------------+
|instId|               value|             topic|
+------+--------------------+------------------+
|  0001|111,sending01,sUs...|institutionId_0001|
+------+--------------------+------------------+

23/05/06 16:21:10 WARN CheckpointFileManager: Failed to rename temp file file:/home/bcturner/instantPaymentProcessingSystem/main/data/central_processor_checkpoints/offsets/.6.314ed759-ab07-452f-89be-cb4cc04b1779.tmp to file:/home/bcturner/instantPaymentProcessingSystem/main/data/central_processor_checkpoints/offsets/6 because file exists
org.apache.hadoop.fs.FileAlreadyExistsException: Rename destination file:/home/bcturner/instantPaymentProcessingSystem/main/data/central_processor_checkpoints/offsets/6 already exists.
	at org.apache.hadoop.fs.AbstractFileSystem.renameInternal(AbstractFileSystem.java:770)
	at org.apache.hadoop.fs.AbstractFileSystem.

StreamingQueryException: Query [id = 6d655502-5f91-4c8d-92d3-a26edb89b489, runId = 452fbff0-51b4-4c94-89c9-3643c523f866] terminated with exception: Multiple streaming queries are concurrently using file:/home/bcturner/instantPaymentProcessingSystem/main/data/central_processor_checkpoints/offsets

-------------------------------------------
Batch: 3
-------------------------------------------
+------+--------------------+------------------+
|instId|               value|             topic|
+------+--------------------+------------------+
|  0001|111,sending01,sUs...|institutionId_0001|
+------+--------------------+------------------+

-------------------------------------------
Batch: 4
-------------------------------------------
+------+--------------------+------------------+
|instId|               value|             topic|
+------+--------------------+------------------+
|  0001|111,sending01,sUs...|institutionId_0001|
+------+--------------------+------------------+

